# Практическое задание: Разработка и оценка рекомендательных систем

Цель:
Разработать различные модели рекомендательных систем и оценить их качество с использованием разных метрик.

Данные:
The Movies Dataset от GroupLens.

Шаги:

Подготовка данных

Загрузите датасет.
Подготовьте данные: создайте матрицу пользователь-фильм на основе рейтингов.
Разработка рекомендательных систем

Content-based рекомендация: Используйте жанры, ключевые слова и другие характеристики фильма для рекомендации.
Collaborative Filtering:
User-based: Используйте схожесть между пользователями.
Item-based: Используйте схожесть между элементами (фильмами).
Matrix Factorization: Используйте SVD.
Оценка качества рекомендательных систем

Разделите данные на обучающую и тестовую выборки.
Используйте следующие метрики для оценки качества каждой модели:
RMSE (Root Mean Squared Error): Используйте для моделей, которые предсказывают рейтинг.
Precision@k и Recall@k: Оцените, сколько рекомендованных элементов действительно интересны пользователю.
Hit Rate: Как часто рекомендация совпадает с предпочтением пользователя.
Diversity: Измеряет насколько разнообразными являются рекомендации.
Выводы

Какая модель показала себя лучше всего по каждой метрике?
Какие сложности и ограничения вы столкнулись при разработке и оценке моделей?
Какие улучшения можно внести в текущие модели?
Дополнительные задачи (для продвинутых пользователей):
- Попробуйте использовать гибридные системы, комбинируя различные методы.
- Исследуйте возможность использования deep learning для создания рекомендаций.
- Исследуйте, как cold start problem (проблема нового пользователя или нового элемента) влияет на каждую из моделей и как с этим можно бороться.

После выполнения этого задания вы получите представление о различных подходах к созданию рекомендательных систем и научитесь оценивать их качество.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import ast


In [2]:
# Загрузка данных
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data = data.dropna(subset='overview')
descriptions = data['overview']



In [3]:
def extract_genres(genres_str):
    genres_list=ast.literal_eval(genres_str)
    genres_cleaned = " ".join([g["name"]for g in genres_list])
    return genres_cleaned

In [4]:
data['genres_cleaned'] = data['genres'].apply(extract_genres)
genres_description = data["genres_cleaned"]

## Реализация метода на основе контента

In [5]:
# 2. Векторизация описаний и жанров
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = tfidf_vectorizer.fit_transform(descriptions)
tfidf_matrix_genres = tfidf_vectorizer.fit_transform(genres_description)

In [6]:
from scipy.sparse import hstack
combined_matrix = hstack([tfidf_matrix,tfidf_matrix_genres])

In [7]:
# 3 Вычисление косинусного сходства
cosine_sim= linear_kernel(combined_matrix, combined_matrix)

In [8]:
# 4. Функция для получения рекомендаций
def get_recommendations(title, n=10):
    idx = data.index[data['title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+1]  # Возвращаем n наиболее похожих фильмов
    movie_indices = [i[0] for i in sim_scores]
    return data['title'].iloc[movie_indices]

In [9]:
# 5. Тестирование системы
print(get_recommendations('The Matrix'))


31746                      That Night
9094                        The Boxer
6204                         The Trip
44075                        The Nude
37017                             Boy
40208             Morris from America
196        The Umbrellas of Cherbourg
18177                    Lebanon, Pa.
9209     Water Drops on Burning Rocks
12333                  Rachel, Rachel
Name: title, dtype: object


In [10]:
#Вычисление Евклидово расстояния
from sklearn.metrics.pairwise import euclidean_distances
euclidean_sim= euclidean_distances(combined_matrix, combined_matrix)

## User-based подход

In [120]:
# Загрузка данных
movies = pd.read_csv('movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('ratings_small.csv')

In [121]:
n_users = ratings['userId'].unique().shape[0]
n_items = ratings['movieId'].unique().shape[0]

In [122]:
ratings.drop(['timestamp'], axis=1, inplace = True)

In [123]:
# Создаем матрицу предпочтений
user_item_matrix =ratings.pivot(index='userId', columns = 'movieId', values = 'rating')
user_item_matrix.fillna(0,inplace=True)

In [124]:
# Делим данные на тренировочный и тестовый наборы
from sklearn.model_selection import train_test_split

train_data,test_data= train_test_split(user_item_matrix,test_size=30)

In [125]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data, metric='cosine')
item_similarity = pairwise_distances(train_data.T, metric='cosine')



In [126]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred
item_prediction = predict(train_data, item_similarity, type='item')
user_prediction = predict(train_data, user_similarity, type='user')


ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))
print ('User based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print ('Item based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))